In [103]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path

path = "/Users/Dipesh/OneDrive/Research_Engineer/Papers/M2EC/results/AWS_all/"
#path = "/Users/Dipesh/OneDrive/Research_Engineer/Papers/M2EC/results/AWS_all_Google/"
cost_path = "/Users/Dipesh/OneDrive/Research_Engineer/Papers/M2EC/results/cost/cost_AWS15.csv"

bandwidth ={}
latency  = {}
bw = []
lt = []

In [104]:
dirList = os.listdir(path)
dirList.remove(".DS_Store")

for i in dirList:
    for j in dirList: 
        if i is j:
            bw.append(0)
            lt.append(0)
            continue
        
        # bandwidth
        if "_aws" in j:
            location = path+"%s/combined_bandwidth_aws_%s_%s.txt"%(i,j,i)
        else:
            location = path+"%s/combined_bandwidth_google_%s_%s.txt"%(i,j,i)
        
        df_bandwidth = pd.read_csv(location, sep='\s+', skiprows=3, nrows=100, header=None)
        headers = ["0", "1", "time","time_unit","transfer","transfer_unit", "bitrate", "bitrate_unit", "retr", "cwnd", "cwnd_unit"]
        df_bandwidth.columns = headers     
        df_bandwidth.drop(['0', '1', 'time', 'time_unit','transfer', 'transfer_unit','cwnd', 'cwnd_unit', 'retr'], axis=1, inplace= True)
        val = df_bandwidth["bitrate"].mean()
        
        bandwidth[j+"_"+i]=val
        bw.append(val)
  
        # latency
        if "_aws" in j:
            location = path+"%s/combined_latency_aws_%s_%s.txt"%(i,j,i)
        else:
            location = path+"%s/combined_latency_google_%s_%s.txt"%(i,j,i)
        
        df_lat = pd.read_csv(location, sep='\s+', skiprows=1, nrows=100, header=None)

        headers_lat = ["0", "1", "2", "3", "4", "5", "latency", "unit"]
        df_lat.columns = headers_lat
        df_lat.drop(['0', '1', '2', '3','4', '5'], axis=1, inplace= True)        
        df_lat["latency"] = df_lat["latency"].map(lambda x:x.replace("time=",""))
        df_lat["latency"] = df_lat["latency"].astype('float')
        #print(df_lat["latency"].max())
        val_lat = df_lat["latency"].mean()
        
        latency[j+"_"+i]=val_lat
        lt.append(val_lat)

In [105]:
df_cost = pd.read_csv(cost_path, sep=';')
df_cost.head()

,from,central_aws,frankfurt_aws,ireland_aws,london_aws,mumbai_aws,northcalifornia_aws,northvirginia_aws,ohio_aws,oregon_aws,paris_aws,saopaulo_aws,seoul_aws,singapore_aws,sydney_aws,tokyo_aws
0,central_aws,0.000,0.020,0.020,0.020,0.02,0.020,0.020,0.020,0.020,0.020,0.020,0.020,0.020,0.020,0.020
1,frankfurt_aws,0.020,0.000,0.020,0.020,0.02,0.020,0.020,0.020,0.020,0.020,0.020,0.020,0.020,0.020,0.020
2,ireland_aws,0.020,0.020,0.000,0.020,0.02,0.020,0.020,0.020,0.020,0.020,0.020,0.020,0.020,0.020,0.020
3,london_aws,0.020,0.020,0.020,0.000,0.02,0.020,0.020,0.020,0.020,0.020,0.020,0.020,0.020,0.020,0.020
4,mumbai_aws,0.086,0.086,0.086,0.086,0.00,0.086,0.086,0.086,0.086,0.086,0.086,0.086,0.086,0.086,0.086


In [106]:
df_cost.drop(['from'], axis = 1, inplace = True)
df_cost.head()

,central_aws,frankfurt_aws,ireland_aws,london_aws,mumbai_aws,northcalifornia_aws,northvirginia_aws,ohio_aws,oregon_aws,paris_aws,saopaulo_aws,seoul_aws,singapore_aws,sydney_aws,tokyo_aws
0,0.000,0.020,0.020,0.020,0.02,0.020,0.020,0.020,0.020,0.020,0.020,0.020,0.020,0.020,0.020
1,0.020,0.000,0.020,0.020,0.02,0.020,0.020,0.020,0.020,0.020,0.020,0.020,0.020,0.020,0.020
2,0.020,0.020,0.000,0.020,0.02,0.020,0.020,0.020,0.020,0.020,0.020,0.020,0.020,0.020,0.020
3,0.020,0.020,0.020,0.000,0.02,0.020,0.020,0.020,0.020,0.020,0.020,0.020,0.020,0.020,0.020
4,0.086,0.086,0.086,0.086,0.00,0.086,0.086,0.086,0.086,0.086,0.086,0.086,0.086,0.086,0.086


In [107]:
# bandwidth
arrayBw = np.array(bw)
# arrayBw= arrayBw.reshape(-1,len(dirList))
arrayBw= arrayBw.reshape(-1,len(dirList))

max_bd = np.max(arrayBw)
min_bd = np.amin(arrayBw[arrayBw>0.00])

# normalization
arrayBw = (arrayBw-min_bd)/(max_bd-min_bd)
arrayBw = arrayBw.round(4)
#arrayBw

In [108]:
# latency
arrayLt = np.array(lt)
arrayLt = arrayLt.reshape(-1,len(dirList))

max_lt = np.max(arrayLt)
min_lt = np.amin(arrayLt[arrayLt>0.00])

# normalization
arrayLt = 1-(arrayLt-min_lt)/(max_lt-min_lt)
arrayLt = arrayLt.round(4)
#arrayLt

In [109]:
# cost
arrayCost = df_cost.values

# Get maximum and minimum
max_cost = np.max(arrayCost) 
min_cost = np.min(arrayCost[arrayCost>0])

# Normalize the values
arrayCost = 1 - (arrayCost - min_cost)/(max_cost - min_cost)
arrayCost = arrayCost.round(4)
#arrayCost

In [110]:
print("Max bandwidth is {}, min bandwidth is {} \nMax latency is {}, min latency is {}\nMax cost is {}, min cost is {}".format(max_bd, min_bd, max_lt, min_lt, max_cost, min_cost))

Max bandwidth is 446.14, min bandwidth is 3.1769 
Max latency is 346.52, min latency is 7.891099999999995
Max cost is 0.16, min cost is 0.01


In [111]:
# convert to 0 for i == j
arrayBw = np.where(arrayBw==-0.0072, 0, arrayBw)
arrayLt = np.where(arrayLt==1.0233, 0, arrayLt)
arrayCost = np.where(arrayCost==1.0667, 0, arrayCost)

In [112]:
# combine latency, bandwidth, and cost with equal weights to all
# combined = (arrayBw+arrayLt+arrayCost)/3
# combined = 0.25 * arrayBw + 0.25 * arrayLt + 0.5 * arrayCost
# combined = 0.2 * arrayBw + 0.2 * arrayLt + 0.6 * arrayCost
# combined = 0.1 * arrayBw + 0.1 * arrayLt + 0.8 * arrayCost
# combined = 0.0 * arrayBw + 0.0 * arrayLt + 1.0 * arrayCost

# combine both latency and bandwidth
# combined = (arrayBw+arrayLt)/2

# only latency
# combined = arrayLt

# only bandwidth
combined = arrayBw

print(combined.shape)

(15, 15)


In [113]:
df = pd.DataFrame(combined)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.0000,0.1723,0.1911,0.0498,0.1010,0.1032,0.5735,0.4071,0.2237,0.1826,0.1703,0.1317,0.1183,0.1206,0.1227
1,0.2163,0.0000,0.5720,0.7060,0.1197,0.0896,0.2776,0.2441,0.1496,0.6994,0.1162,0.1005,0.1534,0.0909,0.1088
2,0.1992,0.4217,0.0000,0.3707,0.1241,0.0984,0.0967,0.2396,0.1881,0.1801,0.1312,0.0796,0.1461,0.0745,0.1145
3,0.1971,0.6508,1.0000,0.0000,0.1017,0.0866,0.2433,0.2395,0.1541,0.8440,0.1204,0.0993,0.1550,0.0901,0.1130
4,0.0888,0.1404,0.1376,0.1399,0.0000,0.0571,0.1103,0.1090,0.0986,0.1661,0.0790,0.1182,0.1965,0.0956,0.0991
5,0.2061,0.1318,0.1355,0.1447,0.0771,0.0000,0.2652,0.3109,0.4694,0.1709,0.1118,0.1421,0.1241,0.1391,0.1596
6,0.6647,0.2403,0.2405,0.3449,0.0809,0.0408,0.0000,0.4666,0.2051,0.1954,0.1719,0.1159,0.0880,0.1138,0.1571
7,0.3801,0.1516,0.1979,0.1788,0.0609,0.1006,0.5602,0.0000,0.2009,0.2284,0.1244,0.0940,0.0854,0.1167,0.1079
8,0.2393,0.1511,0.1657,0.0100,0.0970,0.1914,0.2083,0.2580,0.0000,0.1809,0.1417,0.1615,0.1256,0.1610,0.2116
9,0.1646,0.6777,0.4840,0.8334,0.0864,0.0682,0.2290,0.1767,0.1174,0.0000,0.1055,0.0956,0.1217,0.0938,0.1062


In [114]:
#df = pd.DataFrame(combined, index=dirList, columns=dirList)
df = pd.DataFrame(combined)
df.columns = dirList
new_column = dirList
df.insert(0, column="From", value=new_column)

df = df.round(4)
df

,From,central_aws,frankfurt_aws,ireland_aws,london_aws,mumbai_aws,northcalifornia_aws,northvirginia_aws,ohio_aws,oregon_aws,paris_aws,saopaulo_aws,seoul_aws,singapore_aws,sydney_aws,tokyo_aws
0,central_aws,0.0000,0.1723,0.1911,0.0498,0.1010,0.1032,0.5735,0.4071,0.2237,0.1826,0.1703,0.1317,0.1183,0.1206,0.1227
1,frankfurt_aws,0.2163,0.0000,0.5720,0.7060,0.1197,0.0896,0.2776,0.2441,0.1496,0.6994,0.1162,0.1005,0.1534,0.0909,0.1088
2,ireland_aws,0.1992,0.4217,0.0000,0.3707,0.1241,0.0984,0.0967,0.2396,0.1881,0.1801,0.1312,0.0796,0.1461,0.0745,0.1145
3,london_aws,0.1971,0.6508,1.0000,0.0000,0.1017,0.0866,0.2433,0.2395,0.1541,0.8440,0.1204,0.0993,0.1550,0.0901,0.1130
4,mumbai_aws,0.0888,0.1404,0.1376,0.1399,0.0000,0.0571,0.1103,0.1090,0.0986,0.1661,0.0790,0.1182,0.1965,0.0956,0.0991
5,northcalifornia_aws,0.2061,0.1318,0.1355,0.1447,0.0771,0.0000,0.2652,0.3109,0.4694,0.1709,0.1118,0.1421,0.1241,0.1391,0.1596
6,northvirginia_aws,0.6647,0.2403,0.2405,0.3449,0.0809,0.0408,0.0000,0.4666,0.2051,0.1954,0.1719,0.1159,0.0880,0.1138,0.1571
7,ohio_aws,0.3801,0.1516,0.1979,0.1788,0.0609,0.1006,0.5602,0.0000,0.2009,0.2284,0.1244,0.0940,0.0854,0.1167,0.1079
8,oregon_aws,0.2393,0.1511,0.1657,0.0100,0.0970,0.1914,0.2083,0.2580,0.0000,0.1809,0.1417,0.1615,0.1256,0.1610,0.2116
9,paris_aws,0.1646,0.6777,0.4840,0.8334,0.0864,0.0682,0.2290,0.1767,0.1174,0.0000,0.1055,0.0956,0.1217,0.0938,0.1062


In [117]:
df.to_csv("lt_bd_cost.csv", sep="\t")

In [47]:
new_df = pd.melt(df, id_vars=['From'], value_vars=df.columns.values.tolist()[1:],
    var_name='To', value_name='Value')

In [164]:
new_df.head()

,From,To,Value
0,frankfurt_aws,frankfurt_aws,1.0000
1,frankfurt_google,frankfurt_aws,0.8140
2,london_aws,frankfurt_aws,0.6550
3,london_google,frankfurt_aws,0.5166
4,mumbai_aws,frankfurt_aws,0.3951


In [165]:
new_df.to_csv("values.csv", sep=",")

In [166]:
dirList

['frankfurt_aws',
 'frankfurt_google',
 'london_aws',
 'london_google',
 'mumbai_aws',
 'mumbai_google',
 'northvirginia_aws',
 'northvirginia_google',
 'oregon_aws',
 'oregon_google',
 'saopaulo_aws',
 'saopaulo_google',
 'singapore_aws',
 'singapore_google',
 'sydney_aws',
 'sydney_google',
 'tokyo_aws',
 'tokyo_google']